### Key Rwquirements  
pybind11: A library that connects C++ and python  
numpy: A python library for numerical operations  
An isolated environmet.

In [1]:
# pip install -U pip


/Users/sss/.pyenv/versions/3.10.12/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# Define the CV function
def cv_function(positions, residues, lj_params, alpha, beta):
    energy_matrix = compute_interaction_energy_matrix(positions, residues, lj_params)
    eigenvalues = compute_eigenvalues(energy_matrix)
    eng, sdeng = compute_eng_sdeng(eigenvalues)
    return compute_cv(eng, sdeng, alpha, beta)

# Auto-differentiate CV with respect to atomic positions
compute_cv_gradient = grad(cv_function, argnums=0)

# PyCV Main Function
def compute_cv_with_gradient(positions, frame_idx, residues, lj_params, alpha=1.0, beta=1.0):
    cv_value = cv_function(positions, residues, lj_params, alpha, beta)
    cv_gradient = compute_cv_gradient(positions, residues, lj_params, alpha, beta)
    return cv_value, cv_gradient

But the latest function above doesn't include the alpha and beta modulation.  
The key challenge is that alpha and beta are updated across frames, which makes them stateful variables — something JAX doesn't natively support in grad() because it expects pure functions.

In [ ]:
# Main PyCV Function for PLUMED
def compute_cv_with_modulation(positions, frame_idx, residues, lj_params):
    """
    Compute CV and auto-modulate alpha/beta for PLUMED.
    """
    global eng_series, sdeng_series, alpha, beta

    # Compute interaction energy and eigenvalues
    energy_matrix = compute_interaction_energy_matrix(positions, residues, lj_params)
    eigenvalues = compute_eigenvalues(energy_matrix)

    # Calculate Colvars
    eng, sdeng = compute_eng_sdeng(eigenvalues)

    # Update historical records
    eng_series.append(eng)
    sdeng_series.append(sdeng)

    # Modulate alpha and beta based on history
    percentages = list(range(0, 101, 1))
    alpha, beta = modulate_weights_with_probability(
        eng_series, sdeng_series, frame_idx, alpha, beta, percentages
    )

    # Compute and return the Colvar
    return compute_cv(eng, sdeng, alpha, beta)